<a href="https://colab.research.google.com/github/Martinexe4/Analisis-Sentimen-Aplikasi-Brimo/blob/main/Analisis_Sentimen_Brimo_Apps.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install google-play-scraper


In [ ]:
from google_play_scraper import app, reviews, Sort, reviews_all
import pandas as pd   # Pandas untuk manipulasi dan analisis data
pd.options.mode.chained_assignment = None # Menonaktifkan peringatan chaining
import numpy as np # Numpy untuk operasi numerik dan komputasi
seed = 0
np.random.seed(seed)
import matplotlib.pyplot as plt # Matplotlib untuk visualisasi data
import seaborn as sns # Seaborn untuk visualisasi data lebih baik

import datetime as dt
import re
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords


In [ ]:
!pip install sastrawi

In [ ]:
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory  # Menghapus kata-kata berhenti dalam bahasa Indonesia

from wordcloud import WordCloud  # Membuat visualisasi berbentuk awan kata (word cloud) dari teks

import nltk  # Import pustaka NLTK (Natural Language Toolkit).
nltk.download('punkt')  # Mengunduh dataset yang diperlukan untuk tokenisasi teks.
nltk.download('stopwords')  # Mengunduh dataset yang berisi daftar kata-kata berhenti (stopwords) dalam berbagai bahasa.

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
#Membuat dataframe dari hasil scrapview
app_reviews_df = pd.read_csv('app_reviews.csv')

In [ ]:
jumlah_ulasan, jumlah_kolom = app_reviews_df.shape
print(f"Jumlah baris: {jumlah_ulasan}")
print(f"Jumlah kolom: {jumlah_kolom}")

Jumlah baris: 15000
Jumlah kolom: 11


In [ ]:
app_reviews_df.head()

,reviewId,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt,appVersion
0,096eb4a4-7658-446c-a30b-6256866d3061,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,Menyenangkan,5,0,2.63.0,2024-07-17 22:42:16,NaN,NaN,2.63.0
1,36d817c4-753f-40ad-ba08-8011958d451c,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,👍,4,0,2.63.0,2024-07-17 22:39:57,NaN,NaN,2.63.0
2,030b2d65-1506-4f44-b978-50fabd7ed3f0,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,Mantap,5,0,NaN,2024-07-17 22:35:23,NaN,NaN,NaN
3,1505f41b-7fe8-4aa7-b92f-4394eb35e04b,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,Awalnya bagus tapi hampir sebulan status trans...,1,0,2.63.0,2024-07-17 21:50:05,"Hai Sobat BRI, terima kasih atas ulasannya. Se...",2024-06-16 03:01:05,2.63.0
4,76d31c37-e843-43ed-bb0a-ff3452e32b36,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,Saya mau masuk BRImo selalu pemberitahuan nya ...,2,0,2.63.0,2024-07-17 21:49:02,NaN,NaN,2.63.0


In [ ]:
  app_reviews_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   reviewId              15000 non-null  object
 1   userName              15000 non-null  object
 2   userImage             15000 non-null  object
 3   content               15000 non-null  object
 4   score                 15000 non-null  int64 
 5   thumbsUpCount         15000 non-null  int64 
 6   reviewCreatedVersion  12891 non-null  object
 7   at                    15000 non-null  object
 8   replyContent          12006 non-null  object
 9   repliedAt             12006 non-null  object
 10  appVersion            12891 non-null  object
dtypes: int64(2), object(9)
memory usage: 1.3+ MB


In [ ]:
clean_df = app_reviews_df.dropna()

In [ ]:
clean_df = clean_df.drop_duplicates()

In [ ]:
jumlah_ulasan_setelah_hapus_duplikat, jumlah_kolom_setelah_hapus_duplikat = clean_df.shape
print(f"Jumlah baris setelah menghapus duplikat: {jumlah_ulasan_setelah_hapus_duplikat}")
print(f"Jumlah kolom setelah menghapus duplikat: {jumlah_kolom_setelah_hapus_duplikat}")

Jumlah baris setelah menghapus duplikat: 10394
Jumlah kolom setelah menghapus duplikat: 11


In [ ]:
clean_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10394 entries, 3 to 14999
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   reviewId              10394 non-null  object
 1   userName              10394 non-null  object
 2   userImage             10394 non-null  object
 3   content               10394 non-null  object
 4   score                 10394 non-null  int64 
 5   thumbsUpCount         10394 non-null  int64 
 6   reviewCreatedVersion  10394 non-null  object
 7   at                    10394 non-null  object
 8   replyContent          10394 non-null  object
 9   repliedAt             10394 non-null  object
 10  appVersion            10394 non-null  object
dtypes: int64(2), object(9)
memory usage: 974.4+ KB


In [ ]:
import re
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

def cleaningText(text):
  text = re.sub(r'@[A-Za-z0-9]+', '', text) # menghapus mention
  text = re.sub(r'#[A-Za-z0-9]+', '', text) # menghapus hashtag
  text = re.sub(r'RT[\s]', '', text) # menghapus RT
  text = re.sub(r"http\S+", '', text) # menghapus link
  text = re.sub(r'[0-9]+', '', text) # menghapus angka
  text = re.sub(r'[^\w\s]', '', text) # menghapus karakter selain huruf dan angka

  text = text.replace('\n', ' ') # mengganti baris baru dengan spasi
  text = text.translate(str.maketrans('', '', string.punctuation)) # menghapus semua tanda baca
  text = text.strip(' ') # menghapus karakter spasi dari kiri dan kanan teks
  return text

def casefoldingText(text):
  text = text.lower() # mengubah teks menjadi huruf kecil
  return text

def tokenizingText(text):
  text = word_tokenize(text) #memecah atau membagi string, teks menjadi daftar token
  return text

def filteringText(text):
  listStopwords = set(stopwords.words('indonesian'))
  listStopwords1= set(stopwords.words('english'))
  listStopwords.update(listStopwords1)
  listStopwords.update(['iya','yaa','gak','nya','na','sih','ku',"di","ga","ya","gaa","loh","kah","woi","woii","woy"])
  filtered = []

  for txt in text:
    if txt not in listStopwords:
      filtered.append(txt)
  text = filtered
  return text

def stemmingText(text): # Mengurangi kata ke bentuk dasarnya yang menghilangkan imbuhan awalan dan akhiran atau ke akar kata
    # Membuat objek stemmer
  factory = StemmerFactory()
  stemmer = factory.create_stemmer()

    # Memecah teks menjadi daftar kata
  words = text.split()

    # Menerapkan stemming pada setiap kata dalam daftar
  stemmed_words = [stemmer.stem(word) for word in words]

# Menggabungkan kata-kata yang telah distem
  stemmed_text = ' '.join(stemmed_words)

  return stemmed_text

def toSentence(list_words): # Mengubah daftar kata menjadi kalimat
  sentence = ' '.join(word for word in list_words)
  return sentence


In [ ]:
slangwords = {"@": "di", "abis": "habis", "wtb": "beli", "masi": "masih", "wts": "jual", "wtt": "tukar", "bgt": "banget", "maks": "maksimal"}
def fix_slangwords(text):
  words = text.split()
  fixed_words = []

  for word in words:
    if word.lower() in slangwords:
      fixed_words.append(slangwords[word.lower()])
    else:
      fixed_words.append(word)

  fixed_text = ' '.join(fixed_words)
  return fixed_text

In [ ]:
# Membersihkan teks dan menyimpannya di kolom 'text_clean'
clean_df['text_clean'] = clean_df['content'].apply(cleaningText)

# Mengubah huruf dalam teks menjadi huruf kecil dan menyimpannya di 'text_casefoldingText'
clean_df['text_casefoldingText'] = clean_df['text_clean'].apply(casefoldingText)

# Mengganti kata-kata slang dengan kata-kata standar dan menyimpannya di 'text_slangwords'
clean_df['text_slangwords'] = clean_df['text_casefoldingText'].apply(fix_slangwords)

# Memecah teks menjadi token (kata-kata) dan menyimpannya di 'text_tokenizingText'
clean_df['text_tokenizingText'] = clean_df['text_slangwords'].apply(tokenizingText)

# Menghapus kata-kata stop (kata-kata umum) dan menyimpannya di 'text_stopword'
clean_df['text_stopword'] = clean_df['text_tokenizingText'].apply(filteringText)

# Menggabungkan token-token menjadi kalimat dan menyimpannya di 'text_akhir'
clean_df['text_akhir'] = clean_df['text_stopword'].apply(toSentence)

In [ ]:
clean_df.head(10)

,reviewId,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt,appVersion,text_clean,text_casefoldingText,text_slangwords,text_tokenizingText,text_stopword,text_akhir
3,1505f41b-7fe8-4aa7-b92f-4394eb35e04b,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,Awalnya bagus tapi hampir sebulan status trans...,1,0,2.63.0,2024-07-17 21:50:05,"Hai Sobat BRI, terima kasih atas ulasannya. Se...",2024-06-16 03:01:05,2.63.0,Awalnya bagus tapi hampir sebulan status trans...,awalnya bagus tapi hampir sebulan status trans...,awalnya bagus tapi hampir sebulan status trans...,"[awalnya, bagus, tapi, hampir, sebulan, status...","[bagus, sebulan, status, transfer, proses, tau...",bagus sebulan status transfer proses tau uangn...
19,537e91c1-c3fe-40f5-a57d-c174642ae371,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,"Dengan adanya BRImo BRI, cukup mempermudah tra...",5,0,2.63.0,2024-07-17 19:11:13,Terima kasih atas ulasannya. Semoga aplikasi B...,2024-07-16 10:28:51,2.63.0,Dengan adanya BRImo BRI cukup mempermudah tran...,dengan adanya brimo bri cukup mempermudah tran...,dengan adanya brimo bri cukup mempermudah tran...,"[dengan, adanya, brimo, bri, cukup, mempermuda...","[brimo, bri, mempermudah, transaksi, terima, k...",brimo bri mempermudah transaksi terima kasih b...
46,67ec5424-e5de-4174-a0a9-df89a3ff48d8,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,Bukan mempermudah malah mempersulit ribet daft...,4,0,2.62.0,2024-07-17 16:08:23,"Sobat BRI, mohon maaf atas ketidaknyamanan yan...",2022-07-31 04:53:41,2.62.0,Bukan mempermudah malah mempersulit ribet daft...,bukan mempermudah malah mempersulit ribet daft...,bukan mempermudah malah mempersulit ribet daft...,"[bukan, mempermudah, malah, mempersulit, ribet...","[mempermudah, mempersulit, ribet, daftar]",mempermudah mempersulit ribet daftar
84,64ebcb0e-b2b9-45ac-8c38-fa2883e5d6b5,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,Brimo mantap,5,0,2.63.0,2024-07-17 13:37:42,"Hai Sobat BRI, mohon maaf atas ketidaknyamanan...",2022-11-13 10:02:07,2.63.0,Brimo mantap,brimo mantap,brimo mantap,"[brimo, mantap]","[brimo, mantap]",brimo mantap
96,6c42ae2c-413b-4f1c-817f-9bfbfd76911a,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,BRIMO memang nice👍 sukses terus,5,0,2.63.0,2024-07-17 13:05:46,"Terima kasih sudah menggunakan aplikasi BRImo,...",2024-02-12 22:15:52,2.63.0,BRIMO memang nice sukses terus,brimo memang nice sukses terus,brimo memang nice sukses terus,"[brimo, memang, nice, sukses, terus]","[brimo, nice, sukses]",brimo nice sukses
171,0b58bdca-b29d-41c2-a43f-2a5a956bbbbe,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,"Gk usah down aplikasi ini,SAYA SUDAH BAYAR TRA...",1,0,2.57.0,2024-07-17 09:37:18,"Hai Sobat BRI, kami informasikan transaksi Top...",2022-10-26 04:29:18,2.57.0,Gk usah down aplikasi iniSAYA SUDAH BAYAR TRAN...,gk usah down aplikasi inisaya sudah bayar tran...,gk usah down aplikasi inisaya sudah bayar tran...,"[gk, usah, down, aplikasi, inisaya, sudah, bay...","[gk, aplikasi, inisaya, bayar, transaksi, qris...",gk aplikasi inisaya bayar transaksi qris x k x...
176,fb1b52fd-6dee-4b06-b33a-c6059368b811,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,"Mantaps banget, cuma kok sekarang ribet banget...",5,0,2.63.0,2024-07-17 09:25:26,"Terima kasih sudah menggunakan aplikasi BRImo,...",2022-06-23 05:43:35,2.63.0,Mantaps banget cuma kok sekarang ribet banget ...,mantaps banget cuma kok sekarang ribet banget ...,mantaps banget cuma kok sekarang ribet banget ...,"[mantaps, banget, cuma, kok, sekarang, ribet, ...","[mantaps, banget, ribet, banget, pengen, ganti...",mantaps banget ribet banget pengen ganti nomor...
286,651bea6f-01db-45b8-9a43-efcd7c63abd7,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,Oke..tapi setelah di upgrade sedikit lemot..to...,5,0,2.63.0,2024-07-17 03:28:39,"Sobat BRI, mohon maaf atas ken

In [ ]:
import csv
import requests
from io import StringIO

In [ ]:
lexicon_positive = dict()

# Membaca data kamus kata-kata positif dari GitHub
response = requests.get('https://raw.githubusercontent.com/angelmetanosaa/dataset/main/lexicon_positive.csv')
#Mengirim permintaan HTTP untuk mendapatkan file CSV dari GitHub

if response.status_code == 200:
    # Membaca konten response sebagai teks
    reader = csv.reader(StringIO(response.text), delimiter=',')

    for row in reader:
      # Mengulangi setiap baris dalam file CSV
      lexicon_positive[row[0]] = int(row[1])
      # Menambahkan kata-kata positif dan skornya ke dalam kamus lexicon_positive

else:
    print('Gagal mendapatkan file positive lexicon data dari GitHub')

#Membaca data kamus kata-kata negatif dari GitHub
lexicon_negative = dict()

response = requests.get('https://raw.githubusercontent.com/angelmetanosaa/dataset/main/lexicon_negative.csv')

if response.status_code == 200:
    # Jika permintaan berhasil
    reader = csv.reader(StringIO(response.text), delimiter=',')
    # Membaca teks respons sebagai file CSV menggunakan pembaca CSV dengan pemisah koma

    for row in reader:
        # Mengulangi setiap baris dalam file CSV
        lexicon_negative[row[0]] = int(row[1])
        # Menambahkan kata-kata negatif dan skornya dalam kamus lexicon_negative
else:
    print("Failed to fetch negative lexicon data")

In [ ]:
def sentiment_analysis_lexicon_indonesia(text):

  score = 0

  for word in text:
    if (word in lexicon_positive):
      score += lexicon_positive[word]
    # Jika kata ada dalam kamus positif, tambahkan skornya ke skor sentimen

  for word in text:
    if (word in lexicon_negative):
      score += lexicon_negative[word]
    # Jika kata ada dalam kamus negatif, tambahkan skornya ke skor sentimen

  polarity=''

  if (score >= 0):
        polarity = 'positive'
        # Jika skor sentimen lebih besar atau sama dengan 0, maka polaritas adalah positif
  elif (score < 0):
        polarity = 'negative'
        # Jika skor sentimen kurang dari 0, maka polaritas adalah negatif
  return score, polarity

In [ ]:
results = clean_df['text_stopword'].apply(sentiment_analysis_lexicon_indonesia)
results = list(zip(*results))
clean_df['polarity_score'] = results[0]
clean_df['polarity'] = results[1]
print(clean_df['polarity'].value_counts())

polarity
positive    8057
negative    2337
Name: count, dtype: int64


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam

# Misalkan clean_df sudah didefinisikan sebelumnya
# clean_df = ...

# Pisahkan data menjadi fitur (tweet) dan label (sentimen)
X = clean_df['text_akhir']
y = clean_df['polarity']

# Encode labels (sentimen)
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Membagi data menjadi data latih dan data uji
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Tokenisasi dan padding
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(X_train)

X_train_tokens = tokenizer.texts_to_sequences(X_train)
X_test_tokens = tokenizer.texts_to_sequences(X_test)

max_sequence_length = 100
X_train_pad = pad_sequences(X_train_tokens, maxlen=max_sequence_length)
X_test_pad = pad_sequences(X_test_tokens, maxlen=max_sequence_length)

# Membangun model LSTM
model = Sequential()
model.add(Embedding(input_dim=5000, output_dim=128, input_length=max_sequence_length))
model.add(LSTM(128, return_sequences=True))
model.add(Dropout(0.5))
model.add(LSTM(64))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=0.001), metrics=['accuracy'])

# Melatih model
model.fit(X_train_pad, y_train, epochs=10, batch_size=32, validation_data=(X_test_pad, y_test))

# Evaluasi model
loss, accuracy = model.evaluate(X_test_pad, y_test)
print(f'Accuracy: {accuracy}')


Epoch 1/10
260/260 [==============================] - 83s 299ms/step - loss: 0.3101 - accuracy: 0.8677 - val_loss: 0.1660 - val_accuracy: 0.9399
Epoch 2/10
260/260 [==============================] - 81s 313ms/step - loss: 0.0928 - accuracy: 0.9658 - val_loss: 0.1440 - val_accuracy: 0.9471
Epoch 3/10
260/260 [==============================] - 77s 296ms/step - loss: 0.0410 - accuracy: 0.9880 - val_loss: 0.1403 - val_accuracy: 0.9562
Epoch 4/10
260/260 [==============================] - 80s 306ms/step - loss: 0.0208 - accuracy: 0.9931 - val_loss: 0.1892 - val_accuracy: 0.9505
Epoch 5/10
260/260 [==============================] - 78s 298ms/step - loss: 0.0171 - accuracy: 0.9947 - val_loss: 0.2257 - val_accuracy: 0.9437
Epoch 6/10
260/260 [==============================] - 77s 295ms/step - loss: 0.0166 - accuracy: 0.9941 - val_loss: 0.1960 - val_accuracy: 0.9538
Epoch 7/10
260/260 [==============================] - 84s 324ms/step - loss: 0.0086 - accuracy: 0.9970 - val_loss: 0.2189 - val_ac

In [ ]:
pip freeze requirements.txt

absl-py==1.4.0
accelerate==0.32.1
aiohttp==3.9.5
aiosignal==1.3.1
alabaster==0.7.16
albumentations==1.3.1
altair==4.2.2
annotated-types==0.7.0
anyio==3.7.1
argon2-cffi==23.1.0
argon2-cffi-bindings==21.2.0
array_record==0.5.1
arviz==0.15.1
astropy==5.3.4
astunparse==1.6.3
async-timeout==4.0.3
atpublic==4.1.0
attrs==23.2.0
audioread==3.0.1
autograd==1.6.2
Babel==2.15.0
backcall==0.2.0
beautifulsoup4==4.12.3
bidict==0.23.1
bigframes==1.11.1
bleach==6.1.0
blinker==1.4
blis==0.7.11
blosc2==2.0.0
bokeh==3.3.4
bqplot==0.12.43
branca==0.7.2
build==1.2.1
CacheControl==0.14.0
cachetools==5.4.0
catalogue==2.0.10
certifi==2024.7.4
cffi==1.16.0
chardet==5.2.0
charset-normalizer==3.3.2
chex==0.1.86
click==8.1.7
click-plugins==1.1.1
cligj==0.7.2
cloudpathlib==0.18.1
cloudpickle==2.2.1
cmake==3.27.9
cmdstanpy==1.2.4
colorcet==3.1.0
colorlover==0.3.0
colour==0.1.5
community==1.0.0b1
confection==0.1.5
cons==0.4.6
contextlib2==21.6.0
contourpy==1.2.1
cryptography==42.0.8
cuda-python==12.2.1
cudf-cu12 @ h